In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
import json
import requests 


# "C:\Users\camer\Desktop\7 - GROUP PROJECT\US_Accidents_Dec19.csv"
# "C:\Users\camer\Desktop\7 - GROUP PROJECT\OH accident data.csv"

In [2]:
# Import Accident dataset

accident_data = pd.read_csv(r"C:\Users\camer\Desktop\7 - GROUP PROJECT\US_Accidents_Dec19.csv")

for col in accident_data.columns: 
    print(col) 


ID
Source
TMC
Severity
Start_Time
End_Time
Start_Lat
Start_Lng
End_Lat
End_Lng
Distance(mi)
Description
Number
Street
Side
City
County
State
Zipcode
Country
Timezone
Airport_Code
Weather_Timestamp
Temperature(F)
Wind_Chill(F)
Humidity(%)
Pressure(in)
Visibility(mi)
Wind_Direction
Wind_Speed(mph)
Precipitation(in)
Weather_Condition
Amenity
Bump
Crossing
Give_Way
Junction
No_Exit
Railway
Roundabout
Station
Stop
Traffic_Calming
Traffic_Signal
Turning_Loop
Sunrise_Sunset
Civil_Twilight
Nautical_Twilight
Astronomical_Twilight


In [3]:
# Drop unnecessary columns

reduced_accident_df = accident_data[["ID", "Severity", "Start_Time", "Start_Lat", "Start_Lng", "Street", "County", "State", "Zipcode", 
                                     "Weather_Timestamp", "Temperature(F)", "Wind_Chill(F)", "Humidity(%)", "Pressure(in)", 
                                     "Visibility(mi)", "Wind_Direction", "Wind_Speed(mph)", "Precipitation(in)", "Weather_Condition"]]

df = pd.DataFrame(reduced_accident_df)

df['Start_Time'] = pd.to_datetime(df['Start_Time'])

df['Start_Time'] = df['Start_Time'].apply(lambda x: x.date())

print (df.dtypes)


ID                    object
Severity               int64
Start_Time            object
Start_Lat            float64
Start_Lng            float64
Street                object
County                object
State                 object
Zipcode               object
Weather_Timestamp     object
Temperature(F)       float64
Wind_Chill(F)        float64
Humidity(%)          float64
Pressure(in)         float64
Visibility(mi)       float64
Wind_Direction        object
Wind_Speed(mph)      float64
Precipitation(in)    float64
Weather_Condition     object
dtype: object


In [4]:
# Convert dates(object) to integers

df['Start_Time'] = pd.to_datetime(df['Start_Time'])
print (df.dtypes)



ID                           object
Severity                      int64
Start_Time           datetime64[ns]
Start_Lat                   float64
Start_Lng                   float64
Street                       object
County                       object
State                        object
Zipcode                      object
Weather_Timestamp            object
Temperature(F)              float64
Wind_Chill(F)               float64
Humidity(%)                 float64
Pressure(in)                float64
Visibility(mi)              float64
Wind_Direction               object
Wind_Speed(mph)             float64
Precipitation(in)           float64
Weather_Condition            object
dtype: object


In [28]:
# Limit data to two-year period

start_date = "2017-01-01"
end_date = "2019-01-01"

mask = (df['Start_Time'] > start_date) & (df['Start_Time'] <= end_date)
two_year_df = df.loc[mask]
two_year_df

,ID,Severity,Start_Time,Start_Lat,Start_Lng,Street,County,State,Zipcode,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
9217,A-9218,2,2017-01-02,38.234737,-122.664314,Bodega Ave,Sonoma,CA,94952-1748,2017-01-02 14:15:00,46.4,45.2,76.0,30.02,10.0,NNW,3.5,NaN,Overcast
9218,A-9219,2,2017-01-02,38.246796,-122.627808,E Washington St,Sonoma,CA,94952,2017-01-02 14:15:00,46.4,45.2,76.0,30.02,10.0,NNW,3.5,NaN,Overcast
9219,A-9220,2,2017-01-02,38.192196,-122.543053,Old Lakeville No 2 Rd,Sonoma,CA,94954,2017-01-02 14:35:00,46.4,45.2,81.0,30.03,10.0,NW,3.5,NaN,Overcast
9220,A-9221,3,2017-01-02,37.732292,-122.414909,I-280 S,San Francisco,CA,94110,2017-01-02 12:56:00,52.0,NaN,61.0,30.02,10.0,SSW,15.0,NaN,Overcast
9221,A-9222,3,2017-01-02,38.211773,-122.144753,I-80 E,Solano,CA,94534,2017-01-02 14:54:00,52.0,NaN,66.0,29.98,10.0,SW,12.7,NaN,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899386,A-2899410,2,2017-08-30,34.055160,-118.214070,San Bernardino Fwy E,Los Angeles,CA,90033,2017-08-30 19:47:00,82.0,NaN,49.0,29.68,10.0,Calm,NaN,NaN,Clear
2899387,A-2899411,2,2017-08-30,34.046740,-118.214800,I-5 N,Los Angeles,CA,90033,2017-08-30 19:47:00,82.0,NaN,49.0,29.68,10.0,Calm,NaN,NaN,Clear
2899388,A-2899412,3,2017-08-30,34.031850,-117.135910,I-10 E,San Bernardino,CA,92373,2017-08-30 19:50:00,98.6,NaN,19.0,29.75,6.0,SW,6.9,NaN,Haze
2899389,A-2899413,4,2017-08-30,34.277860,-118.608280,Ronald Reagan Fwy W,Los Angeles,CA,91311,2017-08-30 20:51:00,95.0,NaN,22.0,29.69,10.0,West,3.5,NaN,Clear


In [12]:
# Sorting by time 

sorted_df = df.sort_values(["Start_Time"], ascending=[True])
sorted_df

,ID,Severity,Start_Time,Start_Lat,Start_Lng,Street,County,State,Zipcode,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
9217,A-9218,2,2017-01-02,38.234737,-122.664314,Bodega Ave,Sonoma,CA,94952-1748,2017-01-02 14:15:00,46.4,45.2,76.0,30.02,10.0,NNW,3.5,NaN,Overcast
181777,A-181779,2,2017-01-02,42.912647,-85.628098,28th St SE,Kent,MI,49508,2017-01-02 17:42:00,36.0,28.5,86.0,30.05,3.0,NE,10.4,0.13,Rain
181776,A-181778,2,2017-01-02,42.973274,-85.698257,Garfield Ave NW,Kent,MI,49504,2017-01-02 16:53:00,37.9,34.4,73.0,30.10,6.0,NE,4.6,0.01,Rain
181775,A-181777,2,2017-01-02,43.000839,-83.729286,Brown St,Genesee,MI,48503-3230,2017-01-02 16:53:00,41.0,36.4,65.0,30.10,10.0,East,6.9,NaN,Scattered Clouds
181774,A-181776,3,2017-01-02,43.505161,-83.932877,US-23 S,Saginaw,MI,48604,2017-01-02 16:16:00,39.2,34.2,65.0,30.10,10.0,NNE,6.9,NaN,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147720,A-1147734,3,2019-01-01,37.991882,-122.307938,I-80 E,Contra Costa,CA,94806,2019-01-01 22:53:00,39.9,37.8,49.0,30.19,10.0,South,3.5,NaN,Clear
1147721,A-1147735,3,2019-01-01,37.631657,-122.084389,W Tennyson Rd,Alameda,CA,94545,2019-01-01 22:54:00,42.1,37.1,47.0,30.26,10.0,NE,8.1,NaN,Clear
1147722,A-1147736,2,2019-01-01,38.046963,-122.235771,I-80 W,Contra Costa,CA,94525,2019-01-01 22:53:00,39.9,37.8,49.0,30.19,10.0,South,3.5,NaN,Clear
1147845,A-1147859,3,2019-01-01,34.078262,-117.741730,I-10 W,Los Angeles,CA,91767,2019-01-01 20:47:00,39.2,NaN,31.0,30.18,10.0,Calm,NaN,NaN,Clear


In [26]:
# Sorting by states
state_sorted_df = df.sort_values(["State", 'Start_Time'], ascending=[True, True])
state_sorted_df


,ID,Severity,Start_Time,Start_Lat,Start_Lng,Street,County,State,Zipcode,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
2289159,A-2289183,3,2017-01-02,33.520660,-86.815570,US-78 W,Jefferson,AL,35203,2017-01-02 22:53:00,60.1,NaN,93.0,29.89,10.0,Variable,3.5,NaN,Overcast
2289162,A-2289186,3,2017-01-02,33.527300,-86.801940,US-78 W,Jefferson,AL,35203,2017-01-02 22:53:00,60.1,NaN,93.0,29.89,10.0,Variable,3.5,NaN,Overcast
2289163,A-2289187,3,2017-01-02,33.524829,-86.807129,I-20 W,Jefferson,AL,35203,2017-01-02 22:53:00,60.1,NaN,93.0,29.89,10.0,Variable,3.5,NaN,Overcast
2289761,A-2289785,3,2017-01-04,32.365790,-86.274520,I-85 N,Montgomery,AL,36107,2017-01-04 17:58:00,49.5,NaN,66.0,30.01,10.0,NNW,5.8,NaN,Overcast
2291092,A-2291116,2,2017-01-09,33.496830,-86.815470,I-65 N,Jefferson,AL,35205,2017-01-09 09:53:00,37.9,32.7,32.0,30.67,10.0,Variable,6.9,NaN,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2686937,A-2686961,2,2018-12-22,41.555070,-109.315580,I-80 E,Sweetwater,WY,82901,2018-12-22 10:54:00,19.0,1.6,65.0,30.27,10.0,West,24.2,NaN,Clear
1249439,A-1249454,3,2018-12-26,44.279575,-105.494247,I-90 W,Campbell,WY,82718,2018-12-26 08:53:00,26.1,21.9,88.0,29.99,1.0,WNW,3.5,0.01,Light Snow
1249479,A-1249494,3,2018-12-26,41.633530,-108.379539,I-80 W,Sweetwater,WY,82901,2018-12-26 05:33:00,23.0,NaN,96.0,29.78,1.5,Calm,NaN,0.00,Light Snow
2688488,A-2688512,3,2018-12-26,41.774120,-107.462190,I-80 W,Carbon,WY,82301,2018-12-26 09:44:00,25.0,14.8,81.0,29.83,10.0,NNW,10.4,NaN,Overcast


In [27]:

new_dates = two_year_df['Start_Time'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
new_dates.head()
new_new_dates ['Start_Time'] = new[0]

9217    20170102
9218    20170102
9219    20170102
9220    20170102
9221    20170102
Name: Start_Time, dtype: int32

In [24]:
# Ohio-only data for two-year period
OH_df = two_year_df.loc[two_year_df['State'] == 'OH']
OH_df


,ID,Severity,Start_Time,Start_Lat,Start_Lng,Street,County,State,Zipcode,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition
182192,A-182194,2,2017-01-09,41.722279,-83.536194,N Detroit Ave,Lucas,OH,43612,2017-01-09 07:34:00,19.9,10.1,61.0,30.37,10.0,South,8.1,NaN,Overcast
182287,A-182289,3,2017-01-10,41.721436,-83.501366,I-75 S,Lucas,OH,43612,2017-01-10 07:14:00,33.4,27.3,92.0,29.97,4.0,South,6.9,NaN,Light Snow
182438,A-182440,2,2017-01-11,41.727242,-83.524315,Stickney Ave,Lucas,OH,43612,2017-01-11 08:14:00,32.2,25.8,88.0,30.08,7.0,SSW,6.9,NaN,Clear
182439,A-182441,2,2017-01-11,41.715591,-83.506302,N Matzinger Rd,Lucas,OH,43612,2017-01-11 08:14:00,32.2,25.8,88.0,30.08,7.0,SSW,6.9,NaN,Clear
182725,A-182727,2,2017-01-16,41.728321,-83.493790,Rosemar Rd,Lucas,OH,43611-1013,2017-01-16 06:54:00,28.4,NaN,89.0,30.33,5.0,Calm,NaN,NaN,Overcast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2898980,A-2899004,2,2017-08-30,41.124679,-80.750035,I-80 E,Mahoning,OH,44440,2017-08-30 12:51:00,75.0,NaN,55.0,30.03,10.0,Variable,4.6,NaN,Mostly Cloudy
2898981,A-2899005,2,2017-08-30,41.123653,-80.749362,OH-11 N,Mahoning,OH,44440,2017-08-30 12:51:00,75.0,NaN,55.0,30.03,10.0,Variable,4.6,NaN,Mostly Cloudy
2898993,A-2899017,4,2017-08-30,41.035470,-81.561750,I-277 W,Summit,OH,44314,2017-08-30 15:54:00,78.1,NaN,54.0,29.99,10.0,SSW,5.8,NaN,Mostly Cloudy
2898999,A-2899023,2,2017-08-30,39.980350,-82.961440,I-670 E,Franklin,OH,43219,2017-08-30 16:51:00,80.1,NaN,50.0,30.01,10.0,North,4.6,NaN,Mostly Cloudy
